## Task 3 - Random Pattern n-back Simulation

Author: Alon Loeffler

Required Files/Folders: learning_functions.py | edamame | asn_nw_00350_nj_01350_seed_1581_avl_10.00_disp_01.00_lx_50.00_ly_50.00.mat

Here, we present 7 random patterns to the network, choosing 1 as the target. 
Each drain electrode is associated with a different pattern.
Each pattern is considered 'trained' once the associated output from the drain electrode reaches a threshold, theta.

We then compare outputs from the drain electrodes, and if the target has the highest output current, we say that the 
network correctly classified the information.

If it does not succeed, we increase theta for the target drain, and decrease it for the non-targets.

**NOTE:** This script is used for a single n-back simulation at a time. The Artemis supercomputer was used to run 10 parallel simulations. This is not included in the current repository, but can be requested by emailing aloe8475@sydney.edu.au


## Imports

In [1]:
#Append path to Ruomin's Edamame Package (Nanowire Simulations)
import sys
import os

sys.path.append('../../edamame') #point to edamame locally

#Choose which nw:
nwChoice=700 #350 or 700

if nwChoice == 350:
    fileName='/PhysicalReinforcementLearning/generate_networks/Network Data/asn_nw_00350_nj_01350_seed_1581_avl_10.00_disp_01.00_lx_50.00_ly_50.00'
elif nwChoice == 700:
    fileName='/PhysicalReinforcementLearning/generate_networks/Network Data/asn_nw_00698_nj_02582_seed_002_avl_10.00_disp_01.00_lx_75.00_ly_75.00'

#point to network data path ^ 

saveFig='../../Data/Figures/'  #Define Figure Save location locally 
dataLoc='../../Data/' #Define Data Save Location locally 

#import edamame (neuromorphic nanowire python package by Ruomin Zhu)
from edamame import * 
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import loadmat, savemat
import networkx as nx
from tqdm.notebook import tqdm_notebook as tqdm
from IPython.core.debugger import set_trace

import pickle 
import _pickle as cPickle
import gzip

from learning_functions import genGridNW,point_on_line,dist,getWeightedGraph
from learning_functions import calc_cost,setupStimulus,setupSourcesOnly,runTesting,getNWState,calcOutputs

2022-07-14 14:02:45,739:INFO:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


## Simulation Functions

In [2]:

def buildNetworks(fileName,ManualSources=True,numDrains=2,numSources=9):
    """
    Function to load Networks from  "Test build networks and patterns.ipynb"
    """

    #load data
    nw=loadmat(fileName)
    nwSize = nw['number_of_wires'][0][0]
    nwJunctions=nw['number_of_junctions'][0][0]
    print('Network '+str(nwSize)+ ' Loaded')
#     this_seed=8639
    Network=nw
    connectivity=connectivity__(wires_dict=Network) 

    #fixing file structure from Matlab:
    connectivity.avg_length=connectivity.avg_length[0][0]
    connectivity.number_of_junctions=connectivity.number_of_junctions[0][0]
    connectivity.centroid_dispersion=connectivity.centroid_dispersion[0][0]
    connectivity.dispersion=connectivity.dispersion[0][0]
    # connectivity.generating_number_of_wires=connectivity.generating_number_of_wires[0][0]
    connectivity.gennorm_shape=connectivity.gennorm_shape[0][0]
    connectivity.length_x=connectivity.length_x[0][0]
    connectivity.length_y=connectivity.length_y[0][0]
    connectivity.number_of_wires=connectivity.number_of_wires[0][0]
    connectivity.numOfWires=connectivity.numOfWires[0][0]
    connectivity.numOfJunctions=connectivity.numOfJunctions[0][0]
    connectivity.theta=connectivity.theta[0][0]
    connectivity.this_seed=connectivity.this_seed[0][0]
    
    #find x and y values of each end of each wire 
    xa=connectivity.xa[0]
    xb=connectivity.xb[0]
    ya=connectivity.ya[0]
    yb=connectivity.yb[0]

    #Pick Electrode placement/distance apart:
    
    #DRAINS
#     numDrains=7
    ex=np.zeros(numDrains)*5#*15
    if nwSize > 500:
        eyVal=76
    else:
        eyVal= 51
    ey=np.linspace(-1,eyVal,numDrains)#(-1,101,numDrains)

    elecDrain=genGridNW(xa,xb,ya,yb,ex,ey) #generate drain locations in ex, ey coordinates


    #IN A LINE:
    if nwSize > 500:
        exVal=75
        eyVal = 74
    else:
        exVal=50
        eyVal=49
    ex=np.ones(numSources)*exVal#50
    ey=np.linspace(-2,eyVal,numSources)#(-2,99,numSources)
    
    #IN A 3X3 GRID:
    # middleNWx=int(connectivity.length_x/2)+1
    # middleNWy=int(connectivity.length_y/2)-1
    # distBwElecs=10
    # ex=[middleNWx-distBwElecs,middleNWx-distBwElecs,middleNWx-distBwElecs,middleNWx,middleNWx,middleNWx,middleNWx+distBwElecs,middleNWx+distBwElecs,middleNWx+distBwElecs]
    # ey=[middleNWy-distBwElecs,middleNWy,middleNWy+distBwElecs]*3

    elecSource=genGridNW(xa,xb,ya,yb,ex,ey) #generate source locations in ex, ey coordinates

    #Manual Source Placement:
    if ManualSources:
        if numSources==9:
            if nwSize > 500:
                elecSource=[678, 260, 491, 173, 628, 424, 301, 236, 483] #700nws
            else:
                elecSource=[23,320,194,74, 145, 317, 129, 34, 141] #350 nws

        elif numSources == 4:
            elecSource=[320,42,161,141]

#     elecDrain=elecDrain[[0,-1]]
    
    return connectivity,elecSource,elecDrain,nwSize,nwJunctions

In [3]:
#Generate 9 random 3x3 patterns:
def generatePatterns(numSamples=40,nback=9):
    """
    Function to create Input patterns
    """ 

    pattern=[]
    K = 3
    N = 9

    arr=np.array([1]*K + [0] * (N-K))
    for p in range(nback):
        np.random.shuffle(arr)
        pattern.append(arr.copy())
    target=pattern[0]
    orig_inputs=np.array(pattern).astype('double')
    targets=list(range(1,nback+1))
    
    return orig_inputs,targets

def getNWState_backprop(training_stimulus,train,state,drains,sources,tmpTiming,tmpTargets,limVal,allVals,run_time=2,dt=0.01,disable_tqdm=False):
    """
    This function runs each training epoch and saves the network state at the last timestep of that epoch   
    
    Returns: simulation, network state at end of simulation
    """
    if train:
        #remove non-target drain from simulation
        target = tmpTargets[0]
        drains=drains[target-1]
        print('\nDrain '+str(target)+ ' Only')

    else:
        print('\nAll Drains')
    eles = np.append(drains, sources) #all drains
    #     if len(eles) == num_drain_training + num_source_training:
    training_sim,cost = runSim_backprop(connectivity, 
                                   stimulus = training_stimulus,
                                   junctionMode = 'tunneling',
                                   dt = dt, T = run_time, 
                                   contactMode = 'preSet',
                                   electrodes = eles,
                                   findFirst = False,
                                   start_state = state,
                                   disable_tqdm=disable_tqdm,
                                   collapse=False,tmpTiming=tmpTiming,tmpTargets=tmpTargets,
                                   limVal=limVal,allVals=allVals)  
    JS1 = getJunctionState(training_sim, -1) #save state
    #     else: 
    #         print('Bless you Joel :)')
    return training_sim,JS1   

#Combine drain voltage change with runSim:
def runSim_backprop(Connectivity, 
                    junctionMode='binary', collapse=False,
                    criticalFlux=0.1, maxFlux=1.5e-1,
                    contactMode='farthest', electrodes=None,
                    dt=1e-3, T=10, 
                    stimulus = None,
                    biasType = 'DC',
                    onTime=0, offTime=50000000,
                    onAmp=1, offAmp=0.005,
                    f = 1, customSignal = None,
                    start_state = None,
                    lite_mode = False, save_steps = 1,
                    findFirst = True,
                    disable_tqdm = False,
                    freeze_wire = None, freeze_junction = None, 
                    freeze_TimeStamp = None,tmpTiming=[],tmpTargets=[],
                    limVal=0.5,allVals=[]):


    """
    Function to run neuromorphic network simulation
    
    For the case of multi electrodes, stimulus should be in parameters.
    See help(stimulus__) for how to generate stimulus__ objects.
    """

    SimulationOptions = simulationOptions__(dt = dt, T = T,
                                            connectivity = Connectivity, 
                                            contactMode = contactMode,
                                            electrodes = electrodes)

    if ((contactMode == 'preSet') and (len(electrodes) > 2)) or stimulus is not None:
        from sys import exit
        if stimulus == None:
            logging.warning(f'Multiple electrodes detected. Please input stimulus in parameters!')
            exit()
        if len(stimulus) < len(SimulationOptions.electrodes):
            diff = len(SimulationOptions.electrodes) - len(stimulus)
            for _ in range(diff):
                stimulus.append(stimulus__('Drain', T = T, dt = dt))

        if len(stimulus) > len(SimulationOptions.electrodes):
            logging.warning(f'More stimulus than electrodes. Current setup has {len(SimulationOptions.electrodes)} electrodes!')
            exit()
            
        if len(stimulus[0].signal) < len(SimulationOptions.TimeVector):
            logging.warning(f'Stimulus length not correct, current time vector has length {len(SimulationOptions.TimeVector)}!')
            exit()
        SimulationOptions.stimulus = stimulus
            
    elif contactMode == 'boundary':
        SimulationOptions.stimulus[:int(len(SimulationOptions.electrodes)/2)] = [stimulus__(biasType = biasType, 
                                                                                        T = T, dt = dt,
                                                                                        onTime = onTime, offTime = offTime,
                                                                                        onAmp = onAmp, offAmp = offAmp,
                                                                                        f = f, customSignal= customSignal)
                                                                                        for i in range(int(len(SimulationOptions.electrodes)/2))]
    else:
        SimulationOptions.stimulus[0] = stimulus__(biasType = biasType, 
                                                T = T, dt = dt,
                                                onTime = onTime, offTime = offTime,
                                                onAmp = onAmp, offAmp = offAmp,
                                                    f = f, customSignal= customSignal)
    if start_state == None: 
        JunctionState = junctionState__(Connectivity.numOfJunctions, 
                                    mode = junctionMode, collapse = collapse, 
                                    criticalFlux=criticalFlux, maxFlux = maxFlux)
    else:
        from copy import deepcopy
        JunctionState = deepcopy(start_state)
    
    kwdict = dict()
    if (freeze_wire != None) or (freeze_junction != None):
        kwdict = dict(freeze_wire = freeze_wire,
                    freeze_junction = freeze_junction, 
                    freeze_TimeStamp = freeze_TimeStamp)
        
    this_realization,cost = simulateNetwork_backprop(SimulationOptions, Connectivity, JunctionState,tmpTiming,tmpTargets,limVal,allVals,lite_mode, disable_tqdm, save_steps, **kwdict)
    
    # if we stop simulation at theta, use this code. otherwise we reset voltages to 0 instead
    
#     finalIter=np.max(np.nonzero(this_realization.electrodeCurrent))
#     if finalIter < 199:
#         this_realization.electrodeCurrent=this_realization.electrodeCurrent[:finalIter+1,:]
#         this_realization.junctionVoltage=this_realization.junctionVoltage[:finalIter+1,:]
#         this_realization.junctionConductance=this_realization.junctionConductance[:finalIter+1,:]
#         this_realization.wireVoltage=this_realization.wireVoltage[:finalIter+1,:]
#         this_realization.filamentState=this_realization.filamentState[:finalIter+1,:]
#         this_realization.junctionSwitch=this_realization.junctionSwitch[:finalIter+1,:]
        #final Iter = time , use that to find / save network state
        
        
#         this_realization.TimeVector=this_realization.TimeVector[:finalIter+1,:]

    #     remove zeros from simulations that ended early:
#     this_realization.junctionCurrent
    
    if findFirst:
        from edamame.analysis.GraphTheory import findCurrent
        try:
            activation = findCurrent(this_realization, 1)
            logging.info(f'First current path {activation[0][0]} formed at time = {activation[1][0]} s.')
        except:
            logging.info('Unfortunately, no current path is formed in simulation time.')

    return this_realization,cost

def simulateNetwork_backprop(simulationOptions, connectivity, junctionState,tmpTiming,tmpTargets, limVoltage,allVals,lite_mode = False, disable_tqdm = False, save_steps = 1, **kwargs):
    """
    Function for stepping through each timestep and calculating each node voltage and edge current/conductance
    """
    niterations = simulationOptions.NumOfIterations
    electrodes = simulationOptions.electrodes
    numOfElectrodes = len(electrodes)
    E = connectivity.numOfJunctions
    V = connectivity.numOfWires
    edgeList = connectivity.edge_list
    cost      = []

#     onAmp     = simulationOptions.stimulus[3].onAmp
    
#     maxV      = onAmp *limVoltage
    
    Network = network__()
    sampling = np.arange(0, niterations, save_steps)
    if lite_mode:
        Network.connectivity = connectivity__(adjMat = connectivity.adj_matrix)
        Network.filamentState = np.zeros((int(niterations/save_steps), E))
        Network.wireVoltage = np.zeros((int(niterations/save_steps), V))
        Network.electrodeCurrent = np.zeros((int(niterations/save_steps), numOfElectrodes))
        Network.TimeVector = np.zeros(int(niterations/save_steps))
    else:        
        Network.connectivity = connectivity
        Network.TimeVector = simulationOptions.TimeVector
        Network.filamentState = np.zeros((niterations, E))
        Network.junctionVoltage = np.zeros((niterations, E))
        # Network.junctionResistance = np.zeros((niterations, E))
        Network.junctionConductance = np.zeros((niterations, E))
        Network.junctionSwitch = np.zeros((niterations, E), dtype = bool)
        Network.wireVoltage = np.zeros((niterations, V))
        Network.electrodeCurrent = np.zeros((niterations, numOfElectrodes))

    Network.sources = []
    Network.drains = []
    for i in range(numOfElectrodes):
        if np.mean(simulationOptions.stimulus[i].signal) != 0:
            Network.sources.append(electrodes[i])
        else:
            Network.drains.append(electrodes[i])

    if len(Network.drains) == 0:
        Network.drains.append(electrodes[1])
        
    if 'freeze_wire' in kwargs:
        freeze_wire = kwargs['freeze_wire']
        freeze_TimeStamp = kwargs['freeze_TimeStamp']
    else:
        freeze_TimeStamp = niterations + 1
    
    
    stopTrain=False
    for this_time in tqdm(range(niterations), desc='Running Simulation ', disable = disable_tqdm): #each time step
        if not stopTrain: #if training and reached theta or testing
            this_stimulus = np.array([i.signal[this_time] for i in simulationOptions.stimulus])
#         else:
#             break
        if tmpTiming[this_time]=='Train':
            trainTime=True
            
        elif tmpTiming[this_time]=='Test':

            trainTime=False

        if trainTime and this_time > 0 and not stopTrain: #if we are training, change drain voltages, otherwise just use this_stimulus
            
            this_stimulus[:num_drain_training]=new_stimulus

        #run sim   
        sol = simCore(connectivity, junctionState, this_stimulus, electrodes, simulationOptions.dt)

        if this_time >= freeze_TimeStamp:
            others = np.setdiff1d(range(V), freeze_wire)
            wireVoltage[others] = sol[others]
        else:
            wireVoltage = sol[0:V]

        if lite_mode:
            if this_time%save_steps == 0:
                Network.wireVoltage[this_time//save_steps,:] = wireVoltage
                Network.electrodeCurrent[this_time//save_steps,:] = sol[V:]
                Network.filamentState[this_time//save_steps,:] = junctionState.filamentState
                Network.TimeVector[this_time//save_steps] = simulationOptions.TimeVector[this_time]
        else:
            Network.wireVoltage[this_time,:] = wireVoltage
            Network.electrodeCurrent[this_time,:] = sol[V:]
            Network.filamentState[this_time,:] = junctionState.filamentState
            Network.junctionVoltage[this_time,:] = junctionState.voltage
            Network.junctionConductance[this_time,:] = junctionState.conductance
            Network.junctionSwitch[this_time,:] = junctionState.OnOrOff

        ## BackProp Implementation
        if trainTime:
            n = num_drain_training #num of drains
            y = Network.electrodeCurrent[this_time,:num_drain_training]
        else:
            n = num_drain_testing
            y = Network.electrodeCurrent[this_time,:num_drain_testing]
            
        beta      = 0.05 #learning rate
        grad      = np.zeros((n)) #set gradients to zero for each epoch  
        
        drainCurrents=y #copy of current

        if tmpTargets[this_time] != -2: #skip rest values
            tarIdx=tmpTargets[this_time]
            y = y/(maxCurrent) # normalize current

            d = 1 #allTargets[tarIdx] #define target normalized current
            if trainTime: 
                dsig = this_stimulus[:num_drain_training] #define new drain voltage
            else:
                dsig = this_stimulus[:num_drain_testing]
                
            #calc cost
            cost.append(calc_cost(y,d))

            if trainTime and not stopTrain: #if we are training, update beta
                if drainCurrents[0]>=allVals[tarIdx-1]: #if greater or less than the target current
                    this_stimulus=np.zeros(len(this_stimulus)) #make everything 0V (inputs + drains)
                    stopTrain=True
                else:
                    stopTrain=False 

                
                if not stopTrain:        
                    grad=beta*(y-d) #y = current, d = target #e.g. [y1 vs (d1 = 1), y2 vs (d2 = 0), y3vs (d3 = 0)]

                    #dsig = old voltage drains, dsig2 = new voltage drains, onAmp = input voltage

                    dsig2=np.array(dsig.T+grad)

                    dsig2=np.clip(dsig2,-onAmp*0.9,onAmp*0.9) #set max and min voltages to less than inputs

                    new_stimulus=dsig2

            elif not trainTime:
                new_stimulus=this_stimulus[:num_drain_testing]

    print('Target: 1')
#     print('Voltage: '+ str(new_stimulus))
    
    Network.numOfWires = V
    Network.numOfJunctions = E
    Network.electrodes = simulationOptions.electrodes
    if len(electrodes) <= 2:
        Network.conductance = Network.electrodeCurrent[:,1]/simulationOptions.stimulus[0].signal[sampling]
    if not lite_mode:
        Network.stimulus = [simulationOptions.stimulus[i] for i in range(numOfElectrodes)]
        Network.junctionResistance = 1/Network.junctionConductance
    
#     print(Network.electrodeCurrent[:,1])
    return Network,cost



def runNBACK(onAmp,onAmpTest,dt,Tlen,timesteps,numDT,restDur,nbacks=[],maxCurrent=[1e-5,1e-5],limVal=0.5,accThresh=0.5,increaseVal=0.5,decreaseVal=0.25,accThreshold=False,orderTraining=[],seed=42):
    """
    n-back task:
    input sample 1 with backprop, then sample 2 with backprop, then sample 1 again with reset drain voltages 
    n back here = 1, showing memory
    
    train sample x, test sample x + nback (e.g. nback = 2, sample = [s1 train,s2,s3,s1 test] )

    increase n-back: 
    sample 1 with backprop, sample 2, sample 3... test sample 1 - n=2
    
    Returns: Simulation, currents, training and testing sample orders, thresholds, stimuli
    """  
    np.random.seed(seed)
    
    t           = [[None]*700 for i in range(len(nbacks))]
    simVoltages = []
    saveStim    = []
    count=0
    for nback in nbacks: #how many samples to "remember" in the past before testing
       
        numTestingSamples= numSamples #how many samples across all nback values
                
        #completely random order
        
        orderTrainingLabelsNBACK=[1]*numSamples #these are the testing labels - every i+nback training sample is what we're testing
        orderTesting=[]
        for label in orderTrainingLabelsNBACK: #for each training label in the random order:
            orderTesting.append(-1)

        i = nback
        order=list(orderTraining).copy()
        numInserts=0
        while numInserts < numSamples:
            order.insert(i, -1)
            i += nback+1
            numInserts+=1
        
    
        #remove excess training (i.e. with no testing sample)
        sampleTimes=numDT*(nback+1)
        newTrainingOrder=orderTraining[:sampleTimes-numTestingSamples]

        order=order[:sampleTimes]        
        #RUN SIM
        numClasses          = 9
        signalType          ='DC'
        num_source_training = numSources
        tmpSources          = [[] for i in range(len(order))]
        tmpTargets          = [None]*len(order)
        sim                 = [None]*len(order)
        thresholdValues     = [None]*len(order)
        cost                = [None]*len(order)
        simVoltages         = [None]*len(order)
        accuracy    = []
        timing=[]
        allLabels=[]
        allVals=[limVal]*9
        
        i=0
        numTests = 0
        nwState = None
        for val in tqdm(order): #this method loops through samples in the randomised training order
            
            #FOR EACH SAMPLE:
            
            #are we training or testing?
            if val in orderTesting:
                test=True
                train=False
                newval=0
            elif val in orderTraining:
                test=False
                train=True
                newval=val
                print('Training \n')
                
            #define samples
            if train: #if we're training
                this_sample = inputs[newval].copy()#trainingInputs[val].copy() #xs
                this_label  = targets[newval] #ys
                this_sample[this_sample>0.005]=this_sample[this_sample>0.005]*onAmp #tune voltage based on onAmp
                timing.append('Train')

            elif test: #if we're testing
                this_sample = inputs[newval].copy() #xs
                this_label  = targets[newval] #ys
                this_sample[this_sample>0.005]=this_sample[this_sample>0.005]*onAmpTest #CHANGE THIS TO RAMP:
                timing.append('Test')

            targetTmp = signal_expand(this_label,numDT) #number of timesteps per target
            timing[i] = signal_expand(timing[i],numDT)
        #     #array of nans:
            tmpArray=np.empty(restDur)
            tmpArray[:]=-2
            targetTmp = np.append(targetTmp,tmpArray) #add rest #need to remove targets from rest
            tmpTargets[i]=targetTmp 
            rampV=np.linspace(0,onAmpTest,numDT)
            for j in range(len(this_sample)): #for each source electrode
                tmp = signal_expand(this_sample[j],numDT)
                tmp = np.append(tmp,np.zeros(restDur)) #add rest
                tmpSources[i].append(tmp)
            allLabels.append(this_label)
            stimulus=[]
            if train:
                stimulus.append(stimulus__(biasType='Drain',T=dt*len(tmpSources[i][0]),dt=dt))

            elif test:
                for m in range(num_drain_testing): #create stimulus for each drain electrode
                    stimulus.append(stimulus__(biasType='Drain',T=dt*len(tmpSources[i][m]),dt=dt))

            for m in range(len(tmpSources[i])): #create stimulus for each source electrode
                stimulus.append(stimulus__(biasType='Custom',T=dt*len(tmpSources[i][m]),dt=dt,customSignal=tmpSources[i][m]))
            
            signalLen=Tlen#len(stimulus[0].signal) 
            sim[i],nwState = getNWState_backprop(stimulus,train,nwState,drain_pool,sources,timing[i],tmpTargets[i].astype('int'),limVal,allVals,run_time=int(signalLen*dt),dt=dt,disable_tqdm=False)

            simVoltages[i]=(sim[i].wireVoltage)

            print('Currents: ' + str(sim[i].electrodeCurrent[-1][0]))
            
            if test: #if we are testing
                print('Testing \n')
                #ADRIAN ACCURACY METHOD
                t[count][i]=(calcOutputs(sim[i],sources,drain_pool))
                meanCurrAll=np.mean(t[count][i][0:],axis=1)

                if np.argmax(meanCurrAll)==0:
                    accuracy.append(1)
                else:
                    accuracy.append(0)
                        
                if i >0 and reinforcement: #only use this if we are reinforcing
                    if accuracy[numTests]==0: #if the training didn't work
                        allVals[0]=allVals[0]+increaseVal #update 'max current limit based on reinforcement'
                        allVals[1]=allVals[1]-decreaseVal
                        allVals[2]=allVals[2]-decreaseVal
                        allVals[3]=allVals[3]-decreaseVal
                        allVals[4]=allVals[4]-decreaseVal
                        allVals[5]=allVals[5]-decreaseVal
                        allVals[6]=allVals[6]-decreaseVal
#                         allVals[7]=allVals[7]-decreaseVal
#                         allVals[8]=allVals[8]-decreaseVal
                        
#                 #ALON ACCURACY METHOD
#                 #calucalte accuracy
#                 accThreshold=False
#                 if accThreshold:
#                     #add threshold of difference to ensure accuracy:
#                     argmaxvals=np.argmax([t[count][i][0],t[count][i][1]],axis=0)
#                     for s in range(len(argmaxvals)):
#                         if abs(t[count][i][0][s]-t[count][i][1][s])>1e-6 and argmaxvals[s]==1:
#                             argmaxvals[s]=1
#                         elif abs(t[count][i][0][s]-t[count][i][1][s])<1e-6 and argmaxvals[s] == 1:
#                             argmaxvals[s]=0
# #                     accuracy.append(np.sum(argmaxvals==tmpTargets[testTimes])/len(tmpTargets[testTimes]))
#                     accuracy.append(np.sum(argmaxvals==tmpTargets[i-nback])/len(tmpTargets[i-nback]))
#                 else:
#                     accuracy.append(np.sum(np.argmax([t[count][i][0],t[count][i][1]],axis=0)==tmpTargets[i-nback])/len(tmpTargets[i-nback]))
#                 if i >0:
#                     if accuracy[numTests]<= accThresh: #if the training didn't work
#                         if this_label==0: #drain 1
#                             limVals0=limVals0+increaseVal #update 'max current limit based on reinforcement'
#                             limVals1=limVals1-decreaseVal
#                         elif this_label==1: #drain 2
#                             limVals0=limVals0-decreaseVal
#                             limVals1=limVals1+increaseVal                           
#                 mval1=np.max(np.nonzero(t[count][i][0]))
#                 mval2=np.max(np.nonzero(t[count][i][1]))
#                 print('Final Target Drain Current: ' + str([t[count][i][0][mval1],t[count][i][1][mval2]]))

                print('Currents: ' +str(sim[i].electrodeCurrent[-1][:9]))
                print('Target: ' + str(tmpTargets[i][0]))
                print('Accuracy: '+ str(accuracy[numTests]))
                print("New Target Threshold:" + str(allVals[0])) #just a checking/debugging print
                numTests+=1
            else:
                t[count][i]=(calcOutputs(sim[i],sources,[drain_pool[int(tmpTargets[i][0])-1]]))
#             set_trace()
            thresholdValues[i]=allVals.copy()
            i+=1 
        
        count+=1
    return t,thresholdValues,sim,stimulus,accuracy,tmpTargets,newTrainingOrder,orderTesting,order,testTimes,trainTimes,numTestingSamples

In [5]:
def createTrainingOrder(nback):
    """
    Function to create training order of samples

    """
    seeds=[]
    orderTraining=[]
    shuffle=True
    for n in range(numRuns):   
        tmp=[]
        for i in range(10): #for each sample
            tmp.append(np.random.choice(nback,nback,replace=False))
        newTmp=[]
        tmp2=tmp.copy()

        for i in range(len(tmp)): #for each sample
            for j in range(nback): #for each position
                if shuffle:
                    np.random.shuffle(tmp[i])
                a=tmp[i][j] #find the nth position
                b=np.where(tmp[i]==0) #find the position where the value is 0
                tmp2[i][j]=0 #make the nth position 0
                tmp2[i][b]=a #replace the position which had 0 with the value from the nth position
                newTmp.append(np.array(tmp2[i]))
                np.random.shuffle(newTmp)
        orderTraining.append(np.hstack(np.array(newTmp)))
    orderTraining=orderTraining[0]
    return orderTraining

## Run Simulation

In [15]:
#Parameters
num_drain_training=1
num_drain_testing=7
numSources=9
numSamples=num_drain_testing*10
nbacks         = [num_drain_testing]
nback=nbacks[0]
numRuns        = 1

#CHANGE REINFORCEMENT HERE
reinforcement=False #True (with reinforcement) or False (no reinforcement) 

seed=69
epochsPerSample=int(numSamples/nback)
patternSize=3
 
#Amplitude:
onAmp = 0.3 #Voltage for Training
onAmpTest = 0.1 #Voltage for Testing

#Time Parameters
dt=0.01
Tlen = 200
timesteps   = Tlen/dt
numDT       = 200#int(timesteps/len(inputs)) #number of timesteps per sample
restDur=0

accThreshold=False #do we want to force a minimum (arbitrary) difference between target and non-target drain for accuracy to be counted?


#Build networks
connectivity,sources,drain_pool,nwSize,nwJunctions=buildNetworks(fileName,True,num_drain_testing,numSources)

#Generate inputs
inputs,targets=generatePatterns(numSamples,nback)

# CHOOSE RANDOM ORDER TO PRESENT EACH SAMPLE:
np.random.seed(seed)

b='b0p2' #filament value - #to change b, you must manually change it in simSetting.py in edamame/core/

if not reinforcement: #load training order with reinforcment:
    saveName='698nw_2582nj_working_memory_nback_wReinforcement_7drains_data_'+b #change beta based on simSettings.py

    with open(dataLoc+'/Sim Results/working memory nback/'+saveName+'.pkl', 'rb') as f:
        data=pickle.load(f)
        orderTraining=data[5]

else: #create training order

    orderTraining=createTrainingOrder(nback)
    with open(dataLoc+'/Sim Results/working memory nback/'+saveName+'.pkl', 'wb') as f:
        pickle.dump(orderTraining,f)

#Init Variables
maxCurrent = [1e-5] #target
threshCurrent = [5e-6]
sim            = [None]*numRuns
stimuli        = [None]*numRuns
accuracy       = [None]*numRuns
testTimes      = [None]*numRuns
trainTimes      = [None]*numRuns
numTestingSamples = [None]*numRuns
allLabels      = [None]*numRuns
seeds          =  np.random.choice(10000,size=numRuns,replace=False)

limValues = [threshCurrent[0]]#[1e-6]#list(np.arange(0.05,1.05,0.05))  #if we want to loop through onAmp limit voltage percentage for drains
increaseVal=threshCurrent[0]/6
decreaseVal=threshCurrent[0]/12
accThresh=0.7

loadOnly=False #If we don't want to re-run simulation, change this to True

#Simulation Loop
if loadOnly == False:
    print('Parameters:\n')
    print('Train V: ' + str(onAmp))
    print('Test V: ' + str(onAmpTest))
    print('Pattern: ' + str(patternSize)+'x'+str(patternSize))
    meanAcc=[]
    stdAcc=[]
    outputs=[]
    for val in tqdm(limValues):
        print(val)
        for i in tqdm(range(numRuns)):
            outputs.append(runNBACK(onAmp,onAmpTest,dt,Tlen,timesteps,numDT,restDur,nbacks,maxCurrent,val,accThresh,increaseVal,decreaseVal,accThreshold=accThreshold,orderTraining=orderTraining,seed=seeds[i]))    

Network 698 Loaded
Parameters:

Train V: 0.3
Test V: 0.1
Pattern: 3x3


  0%|          | 0/1 [00:00<?, ?it/s]

5e-06


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/560 [00:00<?, ?it/s]

...

Target: 1
Currents: 0.0
Training 


Drain 7 Only


Running Simulation :   0%|          | 0/200 [00:00<?, ?it/s]

Target: 1
Currents: 0.0

All Drains


Running Simulation :   0%|          | 0/200 [00:00<?, ?it/s]

Target: 1
Currents: 3.1624191218556743e-07
Testing 

Currents: [ 3.16241912e-07  8.57652121e-08  2.88666633e-07  3.29084677e-08
  2.38424110e-08  2.24973457e-07  2.46534446e-07 -4.27329365e-07
 -5.36421890e-06]
Target: 1.0
Accuracy: 1
New Target Threshold:5e-06


### Save Simulations

In [18]:
#SAVE SIM DATA

if reinforcement:
    saveName='698nw_2582nj_working_memory_nback_wReinforcement_7drains_sim_'+b
else:
    saveName='698nw_2582nj_working_memory_nback_noReinforcement_7drains_sim_'+b


#SAVE EACH SIM INDIVIDUALLY
for i in tqdm(range(len(outputs[0][2]))):
    with open(dataLoc+'/Sim Results/working memory nback/'+saveName+str(i)+'.pkl', 'wb') as f:
        pickle.dump(outputs[0][2][i], f) 

  0%|          | 0/560 [00:00<?, ?it/s]

In [19]:
dataNoSim=[outputs[0][0],outputs[0][1],outputs[0][3],outputs[0][4],outputs[0][5],outputs[0][6],outputs[0][7],outputs[0][8],outputs[0][9],outputs[0][10],outputs[0][11],orderTraining]

In [20]:
#SAVE ALL NONSIM DATA
if reinforcement:
    saveName='698nw_2582nj_working_memory_nback_wReinforcement_7drains_data_'+b
else:
    saveName='698nw_2582nj_working_memory_nback_noReinforcement_7drains_data_'+b

#SAVE ALL OTHER DATA TOGETHER
with open(dataLoc+'/Sim Results/working memory nback/'+saveName+'.pkl', 'wb') as f:
        pickle.dump(dataNoSim, f) 